In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets2 import IEX_CLOUD_API_TOKEN

# stocks['Ticker']

In [ ]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

# print(data.status_code)

In [ ]:
price = data['latestPrice']
pe_ratio = data['peRatio']

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        l = yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#        print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,     
            data[symbol]['quote']['latestPrice'],  
            data[symbol]['quote']['peRatio'],     
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True        
        )
        
final_dataframe
            
#    print(symbol)
#    print(data.status_code)

In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True) 

# len(final_dataframe)

In [ ]:
final_dataframe

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio: ")
    while True:
    
        try:
            val = float(portfolio_size)
            return portfolio_size
        except ValueError:
            print("That's not a number! \nTry again: ")
            portfolio_size = input("Enter the value of your portfolio: ")
            continue

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data['AAPL']['advanced-stats']['priceToBook'])

#  Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Deprecation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
print(ev_to_ebitda)
# print(interprise_value/1000000000000)

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns
        ),
        ignore_index = True        
        )
        
rv_dataframe

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

In [ ]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'EV/EBITDA', 'EV/GP',]:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
    
# rv_dataframe.columns

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

In [ ]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile' 
    }

try:

    for metric in metrics.keys():
        for row in rv_dataframe.index:              
            rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
except TypeError:
           
    for metric in metrics.keys():
        for row in rv_dataframe.index:              
            rv_dataframe.loc[row, metrics] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
            rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
    #for metric in metrics.keys():
        #for row in rv_dataframe.index:              
            #rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
        # rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        # stats.percentileofscore(rv_dataframe[metric], value_of_row_column)
        
rv_dataframe

In [ ]:
from statistics import mean
# mean([5, 10])

for row in rv_dataframe.index[:100]:
    value_percentiles = [] 
    for metric in metrics.keys(): 
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
# print(value_percentiles)

rv_dataframe

In [ ]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
len(rv_dataframe.index)

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Number of Shares to Buy'])

rv_dataframe

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
float_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile', percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template], 
                    'N': ['RV Score', percent_template]   
                }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer_save()

In [ ]:
rv_dataframe.loc[row, metrics]